In [ ]:
'''
/*
* author: Yuzhou Chen
* date: 2024-07
* note: The code blocks should not be executed sequentially, but should follow specific manner according to the purpose of simulation.
*/
'''

### **Maxent Simulation**

#### **0. Import Modules**

In [118]:
## necessary R packages ##
## R version 4.4.1 ##
library('dismo')
library(raster)
library(sf)
library(sp)
library(GGally)
library(ggplot2)
library(glue)
library(here)

library(patchwork)  #figure stitching
library(viridis) #color palette

#### **1. Display predictors**

##### 1.1 Read Predictors

In [ ]:
## read predictors ##
# 3 parts: climate factor, DEM, derived factor
tif_folder_paths <- c("E:/Working/Sam/Experiment/Data/Alps/ClimateFactor",
                      "E:/Working/Sam/Experiment/Data/Alps/DEM",
                      "E:/Working/Sam/Experiment/Data/Alps/Derived_Factor")
fnames <- lapply(tif_folder_paths,
function(folder_path) {
  list.files(path = folder_path, 
  pattern = '\\.tif$',
  full.names = TRUE, recursive = TRUE)
})
fnames <- unlist(fnames)
if (length(fnames) == 0) {
  stop("No .tif files found in this folder! Please check the folder path.")
}
# print(fnames)
# Use 'stack' to stack all the grids
predictors <- stack(fnames)
plot(predictors)

##### 1.2 Visualize Predictors

In [ ]:
## visualize predictors ##
save_path = "E:/Working/Sam/Experiment/Figures/Predictors.tif"

# list to store subplots
layer_names <- gsub(pattern = ".*/|\\.tif$", replacement = "", x = fnames)
plots <- list()

# create ggplot objects for each layer
for (i in 1:nlayers(predictors)) {
  layer_df <- as.data.frame(rasterToPoints(predictors[[i]]))
  colnames(layer_df) <- c("x", "y", "value")
  
  p <- ggplot(layer_df, aes(x = x, y = y, fill = value)) +
    geom_tile() +  # use geom_tile to plot raster data
    scale_fill_viridis_c() +
    coord_equal() +
    theme_minimal() +
    labs(title = layer_names[i]) +
    theme(legend.position = "bottom",
    legend.title = element_blank(),
    legend.text = element_text(size = 6),    # legend text size
    legend.key.size = unit(0.3, "cm"),
    plot.title = element_text(size = 9, face = "bold"),
    axis.text.x = element_text(size = 7),
    axis.text.y = element_text(size = 7),
    axis.title.x = element_blank(),
    axis.title.y = element_blank()
) +
    scale_x_continuous(labels = scales::number_format(accuracy = 1, suffix = "°")) +
    scale_y_continuous(labels = scales::number_format(accuracy = 1, suffix = "°"))
  
  # add plot to list
  plots[[i]] <- p
}

# stitch plots together
combined_plot <- wrap_plots(plots, nrow = 3, ncol = 4)

# save plots
ggsave(save_path, plot = combined_plot, dpi = 400, width = 9, height = 8)

##### 1.3 Correlationship of Predictors

In [ ]:
## Correlation analysis ## 
predictors_df <- as.data.frame(values(predictors))
# draw scatterplot matrix
# bar chart(self-defined)
my_bar <- function(data, mapping, ...) {
  ggplot(data, mapping) +
    geom_histogram(color = "blue", fill = "blue", bins = 15)
}

# scatter chart(self-defined)
my_scatter <- function(data, mapping, ...) {
  ggplot(data, mapping) +
    geom_point(color = "red", alpha = 0.2, size = 1.5) +
    geom_smooth(method = "lm", color = "black", ...)
}

# draw ggpairs plot
p <- ggpairs(
  predictors_df,
  lower = list(continuous = wrap(my_scatter)),
  diag = list(continuous = wrap(my_bar)),
  upper = list(continuous = wrap("cor", size = 5))
)

# title
p <- p + ggtitle("Scatterplot matrix for Predictors") +
  theme(
    plot.title = element_text(hjust = 0.5, face = "bold", size = 14),
    # axis.text.x = element_text(angle = 45, hjust = 1, size = 12),
    # axis.text.y = element_text(size = 12),
    # strip.text = element_text(size = 14),
    # text = element_text(size = 12),
    # panel.grid.major = element_line(size = 0.5),
    # plot.margin = margin(1, 1, 1, 1, "cm")
  )

# view the plot
print(p)
ggsave("scatterplot_matrix.png", plot = p, width = 12, height = 12, dpi = 300)

#### **2. Read Presence-Only (Occurence) Data & Bunching for Train**

##### 2.1 **For unbiased/Gauss kernel/Back Thickening**

In [ ]:
## 1、Read species PO points
shp_path <- "E:/Working/Sam/Experiment/Data/Gentiana pannonica Scop/Alps/Alps_Thinned_plant_point_800m.shp"
species_points <- st_read(shp_path)
species_points_df <- as.data.frame(species_points)
# head(species_points_df)

# read occ table
occ <- species_points_df[, c("X", "Y")]

## 2、Witholding a 20% sample for testing (k-fold cross-validation)
fold <- kfold(occ, k=5)
occtrain <- occ[fold != 1, ]
occtest <- occ[fold == 1, ]
# str(occtest)

##### **2.2 For Convex Hull**

##### 1) Visulization: normal

In [ ]:
## 1、Read species PO points
shp_path <- "E:/Working/Sam/Experiment/Data/Gentiana pannonica Scop/Alps/Alps_Thinned_plant_point_800m.shp"
species_points <- st_read(shp_path)
species_points_df <- as.data.frame(species_points)
## 2、Build convex hull
species_points_sf <- st_as_sf(species_points, coords = c("X", "Y"), crs = 4326)
convex_hull <- st_convex_hull(st_union(species_points_sf))
## 3、Convert convex hull to SpatialPolygons
convex_hull_sp <- as(convex_hull, "Spatial")
## 4、Visulize the convex hull and species points
plot(convex_hull_sp, col = 'lightblue')
plot(as(species_points_sf, "Spatial"), add = TRUE, col = 'red')
## 5、Extract species points in convex hull
pin_convex_hull <- st_within(species_points, convex_hull)
pin_convex_hull_logical <- lengths(pin_convex_hull) > 0
species_points_in <- species_points[pin_convex_hull_logical, ]
species_points_in_df <- as.data.frame(species_points_in)
## 6、Witholding a 20% sample for testing (k-fold cross-validation)
occ <- species_points_in_df[, c("X", "Y")]
fold <- kfold(occ, k = 5)
occtrain <- occ[fold != 1, ]
occtest <- occ[fold == 1, ]
str(occtest)

##### 2) Visulization: ggplot2

In [ ]:
## ggplot2 visualization
# convert convex hull to data frame
convex_hull_df <- as.data.frame(st_coordinates(convex_hull))
colnames(convex_hull_df) <- c("longitude", "latitude")

# convert PO points to data frame
species_points_df <- as.data.frame(st_coordinates(species_points_sf))
colnames(species_points_df) <- c("longitude", "latitude")

# visualize
ggplot() +
  geom_point(data = species_points_df, aes(x = longitude, y = latitude), color = 'red', size = 2, alpha = 0.6) +
  geom_polygon(data = convex_hull_df, aes(x = longitude, y = latitude), fill = 'lightblue', alpha = 0.4) +
  coord_equal() +
  labs(title = "Species Occurrence Points within Convex Hull",
       x = "Longitude", y = "Latitude") +
  theme_minimal()

#### **3. Model Settings**

In [135]:
## figure save path ##
save_folder= "E:/Working/Sam/Experiment/Figures/bg points/"

##### Unbiased 2500

In [ ]:
## bg_num=2500 ##
ex_name<-'unbiased_2500'
full_path<-paste0(save_folder,ex_name,".tif")
set.seed(123) # random seed for reproducibility
bg_num <- 2500
bg_points <- randomPoints(predictors, bg_num)
bg_points_df <- as.data.frame(bg_points)
colnames(bg_points_df) <- c("X", "Y")

tiff(filename = full_path, width = 8, height = 6, units = "in", res = 400)
plot(predictors[[11]], main = "Unbiased 2500 Background Points",cex.main = 1.3)
points(bg_points_df$X, bg_points_df$Y, col = 'red', pch = 20,cex=0.5)
points(occ,pch = 1, col = 'black',cex=0.8)

# add legends
legend("bottomright", legend = c("Occurrence Points","Background Points"),
       col = c("black","red"), pch = c(1,18), pt.cex = c(1.5,1.5))
dev.off()

##### Unbiased 5000

In [ ]:
## bg_num=5000 ##
ex_name<-'unbiased_5000'
full_path<-paste0(save_folder,ex_name,".tif")
set.seed(123)
bg_num <- 5000
bg_points <- randomPoints(predictors, bg_num)
bg_points_df <- as.data.frame(bg_points)
colnames(bg_points_df) <- c("X", "Y")

tiff(filename = full_path, width = 8, height = 6, units = "in", res = 400)
plot(predictors[[11]], main = "Unbiased 5000 Background Points")
points(bg_points_df$X, bg_points_df$Y, col = 'red', pch = 20,cex=0.4)
points(occ,pch = 1, col = 'black',cex=0.8)

# add legends
legend("bottomright", legend = c("Occurrence Points","Background Points"),
       col = c("black","red"), pch = c(1,20), pt.cex = c(1.5,1.5))
dev.off()

##### Unbiased 10000

In [ ]:
## bg_num=10000 ##
ex_name<-'unbiased_10000'
full_path<-paste0(save_folder,ex_name,".tif")
set.seed(123)
bg_num <- 10000
bg_points <- randomPoints(predictors, bg_num)
bg_points_df <- as.data.frame(bg_points)
colnames(bg_points_df) <- c("X", "Y")

tiff(filename = full_path, width = 8, height = 6, units = "in", res = 400)
plot(predictors[[11]], main = "Unbiased 10000 Background Points")
points(bg_points_df$X, bg_points_df$Y, col = 'red', pch = 20,cex=0.3)
points(occ,pch = 1, col = 'black',cex=0.8)

# add legends
legend("bottomright", legend = c("Occurrence Points","Background Points"),
       col = c("black","red"), pch = c(1,20), pt.cex = c(1.5,1.5))
dev.off()

##### Biased convex hull 5000

In [ ]:
## covex-hull bg_num=5000 ##
ex_name<-'convex_hull_5000'
full_path<-paste0(save_folder,ex_name,".tif")
set.seed(123)
bg_num <- 5000
bg_points <- randomPoints(predictors, bg_num, ext = convex_hull_sp)
bg_points_df <- as.data.frame(bg_points)
colnames(bg_points_df) <- c("X", "Y")

tiff(filename = full_path, width = 8, height = 6, units = "in", res = 400)
plot(predictors[[11]], main = "Convex Hull 5000 Background Points")
points(bg_points_df$X, bg_points_df$Y, col = 'red', pch = 20,cex=0.3)
points(occ,pch = 1, col = 'black',cex=0.8)

# add legends
legend("bottomright", legend = c("Occurrence Points","Background Points"),
       col = c("black","red"), pch = c(1,20), pt.cex = c(1.5,1.5))
dev.off()

##### Biased Gkernel 5000

In [ ]:
## Target group sampling bg_num=5000 ##
bias_file_path <- "E:/Working/Sam/Experiment/Results/TargetGroupSampling/gkernel_800m_v2.asc"
bias_raster <- raster(bias_file_path)
ex_name<-'Gkernel_5000_v2'
full_path<-paste0(save_folder,ex_name,".tif")

## generate background points based on bias
set.seed(123)
bg_num <- 5000
bias_vals <- getValues(bias_raster)
bias_vals[is.na(bias_vals)] <- 0  # set NA to 0

# use 'prob=' to introduce bias
sample_cells <- sample(1:length(bias_vals), bg_num, prob = bias_vals, replace = TRUE)
bg_coords <- xyFromCell(bias_raster, sample_cells)
bg_points_df <- as.data.frame(bg_coords)
colnames(bg_points_df) <- c("X", "Y")

tiff(filename = full_path, width = 8, height = 6, units = "in", res = 400)
plot(predictors[[11]], main = "Background Points with Target Group Sampling")
points(bg_points_df$X, bg_points_df$Y, col = 'red', pch = 20,cex=0.5)
points(occ,pch = 1, col = 'black',cex=0.8)

# add legends
legend("bottomright", legend = c("Occurrence Points","Background Points"),
       col = c("black","red"), pch = c(1,20), pt.cex = c(1.5,1.5))
dev.off()

##### Biased Background Thickening 5000

In [ ]:
## Background thickening bg_num=5000 ##
ex_name<-'BackgroundThickening_5000'
full_path<-paste0(save_folder,ex_name,".tif")
library(gstat)
## 1. read predictors
temp_tif_folder_path <- c("E:/Working/Sam/Experiment/Data/BTfolder") # With low-resolution to speed up
fnames <- lapply(temp_tif_folder_path, function(folder_path) {
  list.files(path = folder_path, pattern = '\\.tif$', full.names = TRUE, recursive = TRUE)
})
fnames <- unlist(fnames)
if (length(fnames) == 0) {
  stop("No .tif files found in this folder! Please check the folder path.")
}
## 2. Use 'stack' to stack all the grids
predictors_temp <- stack(fnames)
# plot(predictors_temp)

## 3. get var(predictor) names
var_names <- names(predictors_temp)
## 4. calculate the variogram range
variogram_ranges <- numeric(length(var_names))
print("Starting variogram range calculation...")
# loop through each var
for (i in seq_along(var_names)) {
  # get var
  var <- var_names[i]
  # print(paste("Processing variable:", var, "(", i, "of", length(var_names), ")"))
  var_data <- values(predictors_temp[[var]])
  coords <- coordinates(predictors_temp[[var]])
  df <- data.frame(var_data = var_data, x = coords[, 1], y = coords[, 2])

  # remove NA values
  df <- na.omit(df)

  # ensure more than 1 rows
  if (nrow(df) > 1) {
    vgm_model <- variogram(var_data ~ 1, locations = ~ x + y, data = df)
    vgm_fit <- fit.variogram(vgm_model, model = vgm(1, "Sph", NA, NA))
    range <- ifelse(is.na(vgm_fit$range[2]), max(vgm_model$dist), vgm_fit$range[2]) # range[1:3] contains [nugget, range, and sill]
    variogram_ranges[i] <- range
  } else {
    variogram_ranges[i] <- NA
  }
}

# filter the variogram with absolute <=3, and multiply by 10
print(variogram_ranges)
variogram_ranges_filtered <- variogram_ranges[abs(variogram_ranges) <= 3] * 50
print(variogram_ranges_filtered)

# calculate the mean range as 'Thickening radius'
thickening_radius <- mean(variogram_ranges_filtered, na.rm = TRUE)
print(paste("Thickening radius calculated:", thickening_radius))

## --- Finish variogram range calculation --- ##
## --- Start background thickening process --- ##

## 5. create bg points
# bg_num=5000（ratio≈0.65）
set.seed(123)
bg_num <- 8000
# restrict bg points to the extent of the predictor
predictor_mask <- predictors[[5]]
valid_cells <- !is.na(predictor_mask[])
mask_layer <- predictor_mask
mask_layer[!valid_cells] <- NA # set invalid cells to NA
random_coords <- raster::sampleRandom(mask_layer, size = bg_num, xy = TRUE, sp = TRUE)

# calculate the distance from each bg to PO
dist_matrix <- spDists(as.matrix(occtrain),coordinates(random_coords), longlat = TRUE)
# calculate the weights for bg points
weights <- colSums(dist_matrix <= thickening_radius)
# print(weights)
non_zero_count <- sum(weights != 0) # check non-zero count
print(non_zero_count)

# select bg points based on weights
valid_indices <- which(!is.na(weights) & weights > 0)
selected_indices <- sample(valid_indices, size = bg_num, replace = TRUE, prob = weights[valid_indices])
bg_points <- coordinates(random_coords)[selected_indices, 1:2]
bg_points_df <- as.data.frame(bg_points)
colnames(bg_points_df) <- c("X", "Y")


## 6. visualize
tiff(filename = full_path, width = 8, height = 6, units = "in", res = 400)
plot(predictors[[11]], main = "Background Points with Thickening")
points(bg_points_df, col = "red", pch = 20, cex = 0.5)
points(occ, col = "black", pch = 1, cex = 0.8)
# add legends
legend("bottomright", legend = c("Occurrence Points","Background Points"),
       col = c("black","red"), pch = c(1,20), pt.cex = c(1.5,1.5))
dev.off()

#### **4. Model Training**

In [137]:
## figure save path ##
save_folder= "E:/Working/Sam/Experiment/Figures/train/"

In [ ]:
## model training in Alps: shared by all strategies ##
full_path_pre<-paste0(save_folder,ex_name,"_predict.tif")
full_path_roc<-paste0(save_folder,ex_name,"_roc.tif")
plot_title<-'BR 5000'

## Define result folder + Result file name
result_folder <- here("Results",ex_name)
if (!dir.create(result_folder, showWarnings = FALSE)) {
  message("Result folder already exists or could not be created")
}
predict_file<-file.path(result_folder,"maxent_prediction.grd")
eval_file<-file.path(result_folder,"evaluation_results.csv")

## Model Fitting
# 'ex_name' has been defined previously to distinguish methods
me <- maxent(predictors, occtrain, a = bg_points_df,args=c("-J", "-P"),path=result_folder)
# me <- maxent(predictors, occtrain, a = bg_points_df, path=result_folder)
print(me)
# plot(me)
response(me)

## Prediction in Alps
predict_map <- predict(me, predictors, args=c("outputformat=raw"), progress='text', 
     filename=predict_file,overwrite=TRUE)
tiff(filename = full_path_pre, width = 8, height = 6, units = "in", res = 400)
plot(predict_map,main=plot_title)
points(occ,pch = 1, col = 'black',cex=0.8)
# add legends
legend("bottomright", legend = c("Occurrence Points"),
       col = c("black"), pch = c(1), pt.cex = c(1.5))
dev.off()

## Evaluation
bg <- randomPoints(predictors, 5000)
# extract prediction values
pvtest <- data.frame(extract(predictors, occtest))
avtest <- data.frame(extract(predictors, bg))
e1 <- evaluate(me, p=pvtest, a=avtest)
tiff(filename = full_path_roc, width = 6, height = 6, units = "in", res = 400)
plot(e1, 'ROC',main=plot_title)
legend("bottomright", legend = c("Training Data","Random Prediction"),
       col = c("red","grey"), pch = c(15,15), pt.cex = c(1.5,1.5))
dev.off()

# Save evaluation results as csv
eval_results <- data.frame(
  AUC = e1@auc,
  Correlation = e1@cor,
  Kappa = e1@kappa,
  TPR = e1@TPR[1],  
  FPR = e1@FPR[1]  
)
write.csv(eval_results, eval_file, row.names = FALSE)

#### **5. Model Evaluation in Bohemian**

In [ ]:
## model validation in Bohemian: shared by all strategies ##

full_path_valid<-paste0(save_folder,ex_name,"_valid.tif")
# define output file name for new area
predict_file<-file.path(result_folder,"bohemian_prediction.grd")
eval_file<-file.path(result_folder,"bohemian_evaluation.csv")

# predictors in new area
tif_folder_path <- "E:/Working/Sam/Experiment/Data/Bohemian"
fnames <- list.files(path = tif_folder_path, pattern = "\\.tif$", full.names = TRUE)
predictors_new <- stack(fnames)

# occurrence points in new area
shp_path <- "E:/Working/Sam/Experiment/Data/Bohemian/Non_Alps_Thinned_plant_point_800m.shp" 
species_points <- st_read(shp_path)
species_points_df<- as.data.frame(species_points)
occ <- species_points_df[, c("X", "Y")]

# use fitted model to predict in new area
predict_map <- predict(me, predictors_new, args=c("outputformat=raw"), progress='text', 
     filename=predict_file,overwrite=TRUE)
tiff(filename = full_path_valid, width = 6, height = 6, units = "in", res = 400)
plot(predict_map,main=plot_title)
points(occ,pch = 1, col = 'black',cex=0.8)
# add legends
legend("bottomright", legend = c("Real Occurrence Points"),
       col = c("black"), pch = c(1), pt.cex = c(1.5))
dev.off()

# save probabilities on each occ in res_list
res_list <- numeric(length = nrow(occ))
for (i in 1:nrow(occ)) {
  # get corrdinates
  point <- occ[i, ]
  x <- point$X
  y <- point$Y
  # get value on point
  value <- extract(predict_map, cbind(x, y))
  res_list[i] <- value
}
# calculate mean & compare
mean_res <- mean(res_list,na.rm = TRUE)  # mean of occ points
mean_value <- mean(predict_map[], na.rm = TRUE) # mean of whole map
# print result
print(paste("Mean value of occurence points: ",mean_res))
print(paste("Mean value of the whole region: ",mean_value))

criteria = mean_res/mean_value